<a href="https://colab.research.google.com/github/abar-1/SDR-ML-Project/blob/editNN/QAMReceiverV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Function to convert binary data to QAM16 constellations

In [ ]:
pip install tensorflow

16QAM Constellation Modulator

In [27]:
import numpy as np
"""
Used https://dsplog.com/2008/06/01/binary-to-gray-code-for-16qam/ for mappings. When tested, the function works and
the binary is correctly mapped to its correspondingcomplex number based on the constellation. To check, uncomment
the last line in the cell which prints a test run of the function.
"""


def modulator(binary_data, M):
    k = int(np.log2(M))

    # defining the real and imaginary PAM constellation for 16-QAM
    alphaRe = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)
    alphaIm = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)

    # taking b0b1 for real
    ipDecRe = np.array([int(''.join(map(str, b[:k//2])), 2) for b in binary_data])
    ipGrayDecRe = ipDecRe ^ (ipDecRe >> 1)

    # taking b2b3 for imaginary
    ipDecIm = np.array([int(''.join(map(str, b[k//2:])), 2) for b in binary_data])
    ipGrayDecIm = ipDecIm ^ (ipDecIm >> 1)

    # mapping the Gray coded symbols into constellation
    modRe = alphaRe[ipGrayDecRe]
    modIm = alphaIm[ipGrayDecIm]

    # complex constellation
    mod = modRe + 1j * modIm

    return mod

def generate_qam_symbols(M, num_symbols):
    # 4 bits per symbol
    k = int(np.log2(M))

    # Generate random binary data
    random_bits = np.random.randint(0, 2, num_symbols * k)

    # Reshape to match modulator input
    binary_data = random_bits.reshape((-1, k))

    # Modulate Binary Data
    modulated_symbols = modulator(binary_data, M)

    return modulated_symbols, binary_data
#print(generate_qam_symbols(16,5))

def add_awgn(qam_symbols, snr_db):
    # Calculate signal power
    signal_power = np.mean(np.abs(qam_symbols) ** 2)

    # Compute noise power based on SNR (convert dB to linear scale)
    noise_power = signal_power / (10 ** (snr_db / 10))

    # Generate AWGN noise
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*qam_symbols.shape) + 1j * np.random.randn(*qam_symbols.shape))

    # Add noise to the symbols
    noisy_qam_symbols = qam_symbols + noise
    return noisy_qam_symbols

Adding noise to 16QAM signals

In [28]:
import pandas as pd
# Initialize an empty list to store DataFrames
df_list = []

# Getting data of different noise levels (data augmentation to prevent overfitting)
for i in range(15, 55, 5):
    M = 16
    num_symbols = 2000
    qam_symbols, original_binary = generate_qam_symbols(M, num_symbols)

    # Ensure binary data is in a 1D format
    original_binary_flat = [''.join(map(str, bits)) for bits in original_binary]  # Convert each row to a string
    noisy_qam = add_awgn(qam_symbols, i)

    # Create a temporary DataFrame for this iteration
    temp_df = pd.DataFrame({'binary': original_binary_flat, 'complex': noisy_qam})

    # Append the temporary DataFrame to the list
    df_list.append(temp_df)

# Concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(df_list, ignore_index=True)

#Plot Constellations (Before and After Noise)
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,2)
# plt.scatter(noisy_qam.real, noisy_qam.imag, alpha=0.5, label="Noisy")
# plt.title(f"Signal to Noise Ratio = {snr_db} dB)")
# plt.xlabel("In-phase (I)")
# plt.ylabel("Quadrature (Q)")
# plt.grid()

#plt.show()

Build Neural Network

X = Complex

y = Binary

In [29]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

#Target variable (binary data)
y = df['binary'].values
# Convert each element of y to a NumPy array
y = np.array([np.array(list(yi)) for yi in y]) # Convert each element of y to list
#Reshape to 4 bits/symbol (-1 means numpy will figure out how many rows are needed)
y = y.reshape(-1, 4)
y = y.astype(int)

# Manually convert 4-bit binary to integer (0-15)
y = np.array([int("".join(str(bit) for bit in row), 2) for row in y])

# One-hot encoding for 16 classes
y = to_categorical(y, num_classes=16)

# Features (complex data)
X = df['complex'].values
X = np.array([np.array(xi) for xi in X])

# Reshape to (n_samples, 2) for real and imaginary parts
X = [[x.real, x.imag] for x in X]
X = np.array(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(2,)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    # 16 neurons for 16 different possibilities (0000 - 1111)
    Dense(16, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


320/320 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4520 - loss: 1.7962 - val_accuracy: 0.9945 - val_loss: 0.4029
Epoch 2/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9145 - loss: 0.4031 - val_accuracy: 0.9965 - val_loss: 0.0503
Epoch 3/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9626 - loss: 0.1999 - val_accuracy: 0.9973 - val_loss: 0.0193
Epoch 4/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9695 - loss: 0.1379 - val_accuracy: 0.9961 - val_loss: 0.0132
Epoch 5/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9767 - loss: 0.1058 - val_accuracy: 0.9969 - val_loss: 0.0109
Epoch 6/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9768 - loss: 0.0909 - val_accuracy: 0.9977 - val_loss: 0.0084
Epoch 7/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9836 - loss: 0.0685 - val_accuracy: 0.9965 - val_loss: 0.0099
Epoch 8/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9816 - loss: 0.0672 - val_accuracy: 0.9977 - val_

Make NN with last layer w/ 1 neuron

Use same training data

Save Train and Test as CSV



Stage 1: Put the last layer as linear activation, map to 0-15

Stage 2: See if we can use decision trees (what accuracy? how does it compare?)

Stage 3: Increase size of dataset to 20k

In [25]:
import numpy as np

X_pred = np.array([3.3, -2.8])
X_pred = X_pred.reshape(1, -1) # Reshape to have a batch dimension of 1

# Suppress scientific notation for small numbers
np.set_printoptions(suppress=True, precision=4)

predictions = model.predict(X_pred)
print(f"Predictions: {predictions}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Predictions: [[0.     0.     0.     0.     0.     0.     0.     0.     0.9998 0.0002
  0.     0.     0.     0.     0.     0.    ]]
